## Set Up and Load

In [3]:
import pandas as pd
import numpy as np
import re
from pathlib import Path
from sklearn.model_selection import train_test_split

df = pd.read_csv("C:/Projects/04_Fake_News_Detection/data/WELFake_Dataset.csv")
df.head()


,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [4]:
df.shape

(72134, 4)

In [5]:
df.columns

Index(['Unnamed: 0', 'title', 'text', 'label'], dtype='object')

## Quick check for data set

In [6]:
df.isna().sum()

Unnamed: 0      0
title         558
text           39
label           0
dtype: int64

In [7]:
df.duplicated().sum()

np.int64(0)

In [8]:
df["label"].value_counts(dropna=False)

label
1    37106
0    35028
Name: count, dtype: int64

## Keep Only Useful columns + standardize names

In [9]:
# Try to detect columns robustly
col_map = {c.lower().strip(): c for c in df.columns}

title_col = col_map.get("title")
text_col  = col_map.get("text")
label_col = col_map.get("label")

if not all([title_col, text_col, label_col]):
    raise ValueError(f"Could not find required columns. Found: {list(df.columns)}")

df = df[[title_col, text_col, label_col]].rename(columns={
    title_col: "title",
    text_col: "text",
    label_col: "label"
})

df["label"] = pd.to_numeric(df["label"], errors="coerce")
df.head()


,title,text,label
0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,NaN,Did they post their votes for Hillary already?,1
2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


## Clean Text

In [10]:
def clean_text(s):
    if pd.isna(s):
        return ""
    s = str(s).lower()
    s = re.sub(r"http\S+|www\.\S+", " ", s)   # remove urls
    s = re.sub(r"\s+", " ", s).strip()       # normalize spaces
    return s

df["title_clean"] = df["title"].apply(clean_text)
df["text_clean"]  = df["text"].apply(clean_text)

## Create Main feature

In [11]:
df["combined_text"] = (df["title_clean"] + " " + df["text_clean"]).str.strip()

# Keep label numeric + clean
df["label"] = pd.to_numeric(df["label"], errors="coerce")
df[["combined_text", "label"]].head()

,combined_text,label
0,law enforcement on high alert following threat...,1
1,did they post their votes for hillary already?,1
2,unbelievable! obama’s attorney general says mo...,1
3,"bobby jindal, raised hindu, uses story of chri...",0
4,satan 2: russia unvelis an image of its terrif...,1


## Remove unnecessary rows + dupluicates

In [12]:
# remove missing labels
df = df.dropna(subset=["label"])

# remove empty combined text
df = df[df["combined_text"].str.len() > 0]

# drop duplicates based on combined text
df = df.drop_duplicates(subset=["combined_text"]).reset_index(drop=True)

print("After cleaning shape:", df.shape)
print("Label balance:\n", df["label"].value_counts(normalize=True))

After cleaning shape: (63667, 6)
Label balance:
 label
0    0.546421
1    0.453579
Name: proportion, dtype: float64


In [13]:
df["text_length"] = df["combined_text"].str.len()
df["text_length"].describe()

count     63667.000000
mean       3334.195219
std        3619.359138
min           1.000000
25%        1492.000000
50%        2517.000000
75%        4148.000000
max      143035.000000
Name: text_length, dtype: float64

## Split train/test

In [14]:
train_df, test_df = train_test_split(
    df[["combined_text", "label"]],
    test_size=0.2,
    random_state=42,
    stratify=df["label"]
)

## Save the cleaned CSVs

In [18]:
from pathlib import Path
DATA_DIR = Path("C:/Projects/04_Fake_News_Detection/data")

DATA_DIR.mkdir(parents=True, exist_ok=True)
train_path = DATA_DIR / "clean_train.csv"
test_path  = DATA_DIR / "clean_test.csv"

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

print("Saved:", train_path, test_path)
print("Train shape:", train_df.shape, "Test shape:", test_df.shape)

Saved: C:\Projects\04_Fake_News_Detection\data\clean_train.csv C:\Projects\04_Fake_News_Detection\data\clean_test.csv
Train shape: (50933, 2) Test shape: (12734, 2)


## Sanity Check

In [19]:
print("Train label distribution:\n", train_df["label"].value_counts(normalize=True))
print("\nTest label distribution:\n", test_df["label"].value_counts(normalize=True))
train_df.head()


Train label distribution:
 label
0    0.546424
1    0.453576
Name: proportion, dtype: float64

Test label distribution:
 label
0    0.546411
1    0.453589
Name: proportion, dtype: float64


,combined_text,label
359,richest man in the world puts trump in his pla...,1
25894,dave brat: h-1b does not put americans first r...,0
41063,"chinese media alarmed at trump trade adviser, ...",0
16392,trump to lay out what he would do in first 100...,0
15904,linkedin in russland: kommt die sperre? intern...,1
